In [2]:
# Import libraries
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime as dt
import time

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Span, Label, HoverTool, Range1d
from bokeh.embed import components
from bokeh.io import output_notebook

In [3]:
output_notebook()

Loading BokehJS ...

In [5]:
# Retrieve data as pandas dataframe named df
df = web.DataReader('GDPC1', 'fred', dt(2010,1,1))

# Calculate growth rate
df['Y'] = ((1 + (df['GDPC1'] - df['GDPC1'].shift(1))/df['GDPC1'].shift(1))**4 - 1) * 100
df['Y'] = df['Y'].round(1)
# String Date column
df['quarter'] = df.index.quarter
df['year'] = df.index.strftime('%Y')
df['strdate'] = df.year.map(str) + " Q" + df.quarter.map(str)

# Declare variables for chart label
Y = (df['GDPC1'][-1] / 1000).round(decimals=1)
q = df.quarter[-1]
y = df.year[-1]
ch = df['Y'][-1].round(decimals=1)

df = df.dropna()

latest = '{} Q{}: Real GDP: {}T; Growth: {}%'.format(y, q, Y, ch)
xlabel = time.mktime(dt(2010, 7, 3, 2, 0, 0).timetuple())*1000
source = ColumnDataSource({'x': df.index, 'y': df['Y'], 'Date': df.strdate})

def get_width():
    mindate = min(source.data['x'])
    maxdate = max(source.data['x'])
    return 0.8 * (maxdate-mindate).total_seconds()*1000 / len(source.data['x'])

tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=700,height=300,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None,
           toolbar_location="below", toolbar_sticky=False, y_range=Range1d(-3, 6.5))
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.vbar('x', width=get_width(), bottom=0, top='y', source = source, color='Red', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
citation = Label(x=xlabel, y=5, render_mode='css', x_units='data', text_font_size='9pt',
                 text_color='#686868', text=latest, background_fill_color='white', background_fill_alpha=1.0)
p.add_layout(citation)
p.add_tools(HoverTool(tooltips=tooltips, show_arrow=False))
p.toolbar.active_drag = None

script, div = components(p)
show(p)